In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
import os
from pinecone import Pinecone as pcn
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from IPython.display import Markdown
import gradio as gr
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI 

/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### 01 Get API Keys

In [3]:
api_key = os.environ.get("PINECONE_API_KEY")
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# configure client
pc = pcn(api_key=api_key)
index_name = 'recipes-index'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1106}},
 'total_vector_count': 1106}

### 02 Set Embedding Model

In [5]:
embed_model = "text-embedding-3-small"
embed = OpenAIEmbeddings(model=embed_model)

In [6]:
text_field = "description"

vectorStore = PineconeVectorStore(
  index, embed, text_field
)

In [15]:
def get_recipe(query, history=None):# completion llm  
    llm = ChatOpenAI(  
        model_name='gpt-4o-mini',  
        temperature=0.0  
    ) 

    retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    # we can set a similarity score threshold and only return documents with a score above that threshold.
    # search_kwargs={"score_threshold": 0.5}
    # We can also limit the number of documents k returned by the retriever.
    # retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

    # 2. Incorporate the retriever into a question-answering chain.
    system_prompt = (
    """
        Always add a kind and friendly message according to the context at the beginning of the response.
        Take the information from context[0]["metadata"]
        Then follow this format
        Title: [Title of the Recipe]
        Description: [Short Description of the Recipe]
        Preparation Time: [Preparation Time]
        Cooking Time: [Cooking Time]
        Difficulty: [Difficulty Level]
        Serves: [Number of Servings]
        Diet Type: [Diet Type]
        Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
        Ingredients: [List of Ingredients]
        Instructions: [Step-by-step Cooking Instructions]

        If you don't have any general information, just respond with "I don't know!"

        {context}
        {{metadata}}
        """
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    response = rag_chain.invoke({"input": query})
    print(response)
    return response["answer"]

## Gradio App

In [16]:
demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


{'input': 'Recommend a recipe with chicken', 'context': [Document(id='982', metadata={'calories': '502', 'carbs': '9g', 'cook_time': '40 mins', 'diet_type': 'Egg-free', 'difficulty': 'Easy', 'fat': '30g', 'fibre': '1g', 'id': '982', 'ingredients': '4chicken breasts, skin left on, 2-3 tbspolive or rapeseed oil, 1onion, finely chopped, 3garlic cloves, crushed or finely grated, 1 tbspdried mixed herbs, 1 tbspplain flour, 200mlwhite wine, 250mlchicken stock, 125mldouble cream, choppedparsley, to serve (optional)', 'instructions': 'Season the chicken all over with salt and freshly ground black pepper. Heat 2 tbsp of the oil in a large lidded frying pan over a medium heat and fry the chicken, skin-side down, for 8-10 mins until the skin is golden. Turn the chicken over and cook for 6-8 mins more until golden all over. Remove to a plate and set aside. If the pan is dry, drizzle in another 1 tbsp oil, then reduce the heat to medium-low and fry the onions for 8-10 mins until softened but not go

## Test History

In [37]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Inicializando el modelo y el retriever
llm = ChatOpenAI(  
    model_name='gpt-4o-mini',  
    temperature=0.0  
)

retriever = vectorStore.as_retriever(search_type="similarity")

# Prompt para contextualizar la pregunta
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [52]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import AIMessage, HumanMessage


def get_recipe(query, history=None):
    # Inicializa el historial de chat si es la primera vez
    history = []    # Prompt del sistema que guía cómo generar la respuesta
    
    system_prompt = (
        """
        Always add a kind and friendly message according to the context at the beginning of the response.
        Take the information from context[0]["metadata"]
        Then follow this format
        Title: [Title of the Recipe]
        Description: [Short Description of the Recipe]
        Preparation Time: [Preparation Time]
        Cooking Time: [Cooking Time]
        Difficulty: [Difficulty Level]
        Serves: [Number of Servings]
        Diet Type: [Diet Type]
        Nutrition: calories context[0]["metadata"]["calories"] | fat context[0]["metadata"]["fat"] | protein context[0]["metadata"]["protein"] | fiber context[0]["metadata"]["fibre"]
        Ingredients: [List of Ingredients]
        Instructions: [Step-by-step Cooking Instructions]

        If you don't have any general information, just respond with "I don't know!"

        {context}
        {{metadata}}
        """
    )

    # Configura el prompt del QA (sistema y humano)
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    # Crear las cadenas para el QA y el RAG (retrieval-augmented generation)
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    # Invoca la cadena con la pregunta y el historial de chat
    response = rag_chain.invoke({"input": query, "chat_history": history})

    # Actualiza el historial de chat con los nuevos mensajes
    history.extend(
        [
            HumanMessage(content=query),
            AIMessage(content=response["answer"]),
        ]
    )

    # Imprimir el mensaje generado por la IA
    print("####### ai_msg_1:", response)

    # Devolver el contenido del mensaje de la IA
    return response["answer"]

In [53]:
demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli", "please recommend 3 recipes with rice"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


####### ai_msg_1: {'input': 'What can I do with brocolli', 'chat_history': [HumanMessage(content='What can I do with brocolli'), AIMessage(content='Hello! Broccoli is such a versatile vegetable, and there are many delicious ways to prepare it. Here are a few ideas:\n\n1. **Cheesy Broccoli Soup**: Use frozen cauliflower and broccoli to make a creamy soup topped with a cheesy, seedy crumble for added texture.\n\n2. **Purple Sprouting Broccoli Salad**: Make a vibrant salad with purple sprouting broccoli, dressed in a tangy vinaigrette made from capers, parsley, mustard, and honey.\n\n3. **Broccoli Stalk Falafel**: Reduce food waste by using broccoli stalks in a falafel recipe, blending them with chickpeas for a tasty and sustainable dish.\n\n4. **Umami Broccoli Noodles**: Create a bowl of noodles featuring broccoli, enhanced with a crispy chili oil topping for an extra kick.\n\nIf you need specific recipes for any of these ideas, feel free to ask!')], 'context': [Document(id='1102', metad